In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [2]:
utils_ops.tf = tf.compat.v1
tf.gfile = tf.io.gfile

In [3]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [4]:
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [5]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[WindowsPath('object_detection/test_images/image1.jpg'),
 WindowsPath('object_detection/test_images/image2.jpg'),
 WindowsPath('object_detection/test_images/image3.jpg'),
 WindowsPath('object_detection/test_images/IMG_1129.JPG')]

In [6]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [7]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  input_tensor = tf.convert_to_tensor(image)
  input_tensor = input_tensor[tf.newaxis,...]

  output_dict = model(input_tensor)

  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  if 'detection_masks' in output_dict:
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [8]:
def show_inference(model, image_path):
    image_np = image_path
    image_np=cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)
    output_dict = run_inference_for_single_image(model, image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=2)
    image_np=cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)
    bicycle_detect(image_np,output_dict['detection_classes'],output_dict['detection_scores'],output_dict['detection_boxes'])
    return image_np

In [9]:
from datetime import datetime

def bicycle_detect(image,classes,score,boxes):
    
    global vahicalName
    
    for i in range (10):
        if(classes[i]==3  and score[i]>0.6):
            vahicalName="CAR"
            print("car")
        elif(classes[i]==4  and score[i]>0.6):
            vahicalName="motorbike"
            print("motorbike")
        elif(classes[i]==6  and score[i]>0.6):
            vahicalName="BUS"
            print("bus")
        else:
            vahicalName="No Vehicles"
            
            

In [10]:
import cv2
import numpy as np
import tkinter
from threading import Thread, Event

import pyfirmata
import time
from pyfirmata import Arduino,util,STRING_DATA
import pyttsx3

textSpeech= pyttsx3.init()
vahicalName="No Vehicles"

x=100
distance=0
xx=0
event = Event()
source=cv2.VideoCapture(r'F:\research full codes\Samples\Comp 6.mp4')
#source=cv2.VideoCapture(0)
width=480
fcx=width//2


audioStatus=False


kernel=np.array([[-1,0,1],
                 [-2,0,2],
                 [-1,0,1]],np.float)




port='COM6'                     #Arduino ports

board = pyfirmata.Arduino('COM6')
led1 = board.get_pin('d:11:o')
led2 = board.get_pin('d:12:o')
led3 = board.get_pin('d:13:o')

stop_threads=False




def modify_variable(var):
    global distance
    global vahicalName
    cop=distance
    while True:
        
        ret,imgs=source.read()
        img=cv2.resize(imgs,(480,340))
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        blur=cv2.blur(gray,(5,5))

        conv=cv2.filter2D(blur,-1,kernel)


        img[:,fcx]=(0,255,0)


        for i in range(240):
            for j in range(240-(i),0,-1):
                conv[i,j]=0
                gray[i,j]=0
                #print("*", end="")
       # print()
    
        ret, thresh = cv2.threshold(conv, 50, 255, 0)
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            area=cv2.contourArea(cnt)
            #print(area)
            if(area>300 and area>310):
                cv2.drawContours(img, [cnt], -1, (0,0,255), 3)            
                rect = cv2.minAreaRect(cnt)
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                cv2.drawContours(img,[box],0,(0,255,0),2)


                M=cv2.moments(cnt)
                cx = int(M['m10']/M['m00'])
                cy = int(M['m01']/M['m00'])
                cv2.circle(img,(cx,cy),5,(0,255,255),-1)

                if(cx<fcx and cy>150):
                    cv2.line(img,(cx,cy),(fcx,cy),(255,0,0,),2)
                    cop=fcx-cx
                    cv2.putText(img,"distance: "+str(cop),(fcx,170),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
                    xx=cop

                #if(distance<=150):
                        #print ("Distance is : "+str(distance))

        img=show_inference(detection_model,img)
        #print(detection_model)
        cv2.imshow('LIVE',img)

        if(distance<=100): #blinking LED
            led1.write(1)
            led2.write(0)
        else:
            led1.write(0)
            led2.write(1)
        
        board.send_sysex(STRING_DATA,util.str_to_two_byte_iter("Distance is "+str(distance)))
        board.send_sysex(STRING_DATA,util.str_to_two_byte_iter(vahicalName))
        
        #print(cop)
        distance=cop
        #cv2.imshow('thresh',thresh)
        #cv2.imshow('gray',gray)
        key=cv2.waitKey(100)

        if(key==27):
            global audioStatus
            audioStatus = False
            break
        
        
    cv2.destroyAllWindows()
    source.release()
        
        
    
    
t = Thread(target=modify_variable, args=(distance, ))
t.start()


window=tkinter.Tk()

def audioCl(): 
    global audioStatus
    audioStatus = True
    t2 = Thread(target=audio, args=(distance, ))
    t2.start()


def audioStop(): 
    global audioStatus
    audioStatus = False

def audio(dis):               ############ Audio Reading
    
    global audioStatus
    global vahicalName
    while(audioStatus):
        global distance
        text='distance is ' + str(distance) + 'and behind ' + vahicalName
        textSpeech.say(text)
        textSpeech.runAndWait()
        
        time.sleep(1)


    



def callcl():
    t1 = Thread(target=cl, args=(distance, ))
    t1.start()






def cl(some):
    global distance
    num=0
    
    
    while(True):
        #num=num+1
        #print("heloooo")
        distanceNew=tkinter.Label(window,text=str(distance),padx='10',font=("Arial Bold",25),background="#004E60",foreground="white")
        distanceNew.place(x=420,y=140)
        if (distance<=100):
            #print('sss')
            emergency1=tkinter.Label(window,text="RISK ",padx='10',font=("Arial Bold",25),background="#F90808",foreground="white")
            emergency1.place(x=900,y=200)
            window.update_idletasks()
        else:
            emergency1=tkinter.Label(window,text="SAFE",padx='10',font=("Arial Bold",25),background="#19FC06",foreground="white")
            emergency1.place(x=900,y=200)
            window.update_idletasks()
            
        
        
        window.update_idletasks()



window=tkinter.Tk()
window.title("Mission")
window.geometry('1500x650')
window.configure(bg='#004E60')
title=tkinter.Label(window,text="MotorBick Assistent Tool",padx='250',font=("Arial Bold",50),background="#004E60",foreground="white")
title.pack()
midLine=tkinter.Label(window,text="Disane to Middle Line :",padx='10',font=("Arial Bold",25),background="#004E60",foreground="white")
distanceNew=tkinter.Label(window,text="0",padx='10',font=("Arial Bold",25),background="#004E60",foreground="white")

behindVehicals=tkinter.Label(window,text="Behind Vehicals :",padx='10',font=("Arial Bold",25),background="#004E60",foreground="white")
lineDetect=tkinter.Label(window,text="one line",padx='10',font=("Arial Bold",25),background="#FFFF00",foreground="black")
vehicalsNew=tkinter.Label(window,text="0",padx='10',font=("Arial Bold",25),background="#004E60",foreground="white")

emergency1=tkinter.Label(window,text="SAFE",padx='10',font=("Arial Bold",25),background="#19FC06",foreground="white")
emergency2=tkinter.Label(window,text="RISK",padx='10',font=("Arial Bold",25),background="#F90808",foreground="white")



#buttons

startBtn= tkinter.Button(window,text='Enter',font=("Arial Bold",25),bg="#100BF7",fg="white",borderwidth = '4',activebackground="red"
                        ,bd='7',relief='raised',command=callcl)

audioStrtBtn= tkinter.Button(window,text='Start Audio',font=("Arial Bold",25),bg="#100BF7",fg="white",borderwidth = '4',activebackground="red"
                        ,bd='7',relief='raised',command=audioCl)

audioStopBtn= tkinter.Button(window,text='Stop Audio',font=("Arial Bold",25),bg="#100BF7",fg="white",borderwidth = '4',activebackground="red"
                        ,bd='7',relief='raised',command=audioStop)



midLine.place(x=10,y=140)
startBtn.place(x=1200,y=20)
audioStrtBtn.place(x=1200,y=100)
audioStopBtn.place(x=1200,y=180)
emergency1.place(x=900,y=200)
lineDetect.place(x=1200,y=200)
behindVehicals.place(x=10,y=400)
emergency2.place(x=900,y=400)
distanceNew.place(x=420,y=140)
vehicalsNew.place(x=420,y=400)

window.mainloop()
        

bus
bus
bus
bus
motorbike
motorbike
motorbike
motorbike


Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\gihan\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\gihan\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-3160d2fd8fa6>", line 191, in cl
  File "C:\Users\gihan\anaconda3\lib\tkinter\__init__.py", line 1320, in update_idletasks
    self.tk.call('update', 'idletasks')
_tkinter.TclError: can't invoke "update" command: application has been destroyed
